<a href="https://colab.research.google.com/github/notwld/chatbot/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import nltk
nltk.download("punkt")

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import tensorflow as tf
import numpy as np
import random,json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
# Using google colab's files method to upload json file which contains the data 

# from google.colab import files
# files.upload()


In [27]:
with open("intents.json",'r') as data:
  intents = json.load(data)

In [28]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Mayank.',
    'Thanks for asking. I am designed by Mayank Bajaj.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ["We are from Worl

In [29]:
words = []
classes = []
documents = []
ignore = ["?"]

In [30]:
# data preprocessing: looping through json file which is saved in variable 
# intents and tokenizing the patterns, extract word with it's tag and saving 
# them in the list.

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    word = nltk.word_tokenize(pattern)
    words.extend(word)

    documents.append((word,intent["tag"]))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])



In [39]:
words = [stemmer.stem(word.lower()) for word in words if word not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))


In [40]:
print("Total documents:", len(documents))
print("Total classes:", len(classes))
print("Total unique words:", len(words))

Total documents: 27
Total classes: 8
Total unique words: 52


In [41]:
# creating lists for training the data

train = [] # f(x)
output = [] # f(y)

output_empty = [0] * len(classes)

In [42]:
#test

documents[0]

(['Hi'], 'greeting')

In [44]:
for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
  for w in words:
    bag.append(1) if word in pattern_words else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])]=1

  train.append([bag,output_row])

random.shuffle(train)

train = np.array(train)

x = list(train[:,0])
y = list(train[:,1])


<ipython-input-44-719e020afe1d>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train = np.array(train)


In [48]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(y[0]),activation="softmax"))

model.compile(tf.keras.optimizers.Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [49]:
model.fit(np.array(x),np.array(y),epochs=100,batch_size=8,verbose=1)
model.save("model.pkl")

Epoch 1/100
4/4 [==============================] - 1s 7ms/step - loss: 2.0811 - accuracy: 0.0370
Epoch 2/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0792 - accuracy: 0.1481
Epoch 3/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0786 - accuracy: 0.1481
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0787 - accuracy: 0.1852
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0780 - accuracy: 0.1852
Epoch 6/100
4/4 [==============================] - 0s 5ms/step - loss: 2.0775 - accuracy: 0.1852
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0776 - accuracy: 0.1852
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0766 - accuracy: 0.1852
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: 2.0766 - accuracy: 0.1852
Epoch 10/100
4/4 [==============================] - 0s 4ms/step - loss: 2.0762 - accuracy: 0.1852
Epoch 11/100
4/4 [===========